In [44]:
import numpy as np
import pickle
import plotly.express as px
from scipy import stats

# Loading the data

## Specifying gesture

In [2]:
gesture_name = "click"
subjects = [f"Subject{i}" for i in range(1, 6)]

## 50 cm data

In [3]:
data_50 = list()

for subject in subjects:
    with open(f"data/randomized/{subject}/50cm/{gesture_name}", "rb") as file:
        temp = pickle.load(file)
        data_50 += temp

## 75 cm data

In [4]:
data_75 = list()

for subject in subjects:
    with open(f"data/randomized/{subject}/75cm/{gesture_name}", "rb") as file:
        temp = pickle.load(file)
        data_75 += temp

## 100 cm data

In [5]:
data_100 = list()

for subject in subjects:
    with open(f"data/randomized/{subject}/100cm/{gesture_name}", "rb") as file:
        temp = pickle.load(file)
        data_100 += temp

# Example gesture

In [6]:
landmark = data_50[0]

xs = [landmark[i][0] for i in range(len(landmark))]
ys = [landmark[i][1] for i in range(len(landmark))]

px.scatter(x=xs, y=ys)

In [18]:
landmark = data_100[0]

xs = [landmark[i][0] for i in range(len(landmark))]
ys = [landmark[i][1] for i in range(len(landmark))]

px.scatter(x=xs, y=ys)

# Normalizing the landmarks using MinMax Scaling

In [7]:
def normalize(z: float, max: float, min: float) -> float:
    norm = (z - min)/(max - min)
    round_norm = round(norm, 3)
    
    return round_norm

In [8]:
def normalize_landmarks(landmarks: list) -> list:
    norm_landmarks = list()

    for landmark in landmarks:
        xs = [landmark[i][0] for i in range(len(landmark))]
        ys = [landmark[i][1] for i in range(len(landmark))]

        xmax = max(xs)
        ymax = max(ys)
        xmin = min(xs)
        ymin = min(ys)

        norm_landmark = list()

        for x, y in zip(xs, ys):
            norm = [normalize(z=x, max=xmax, min=xmin), normalize(z=y, max=ymax, min=ymin)]

            norm_landmark.append(norm)

        norm_landmarks.append(norm_landmark)

    return norm_landmarks

In [9]:
norm_50  = normalize_landmarks(landmarks=data_50)
norm_75  = normalize_landmarks(landmarks=data_75)
norm_100 = normalize_landmarks(landmarks=data_100)

# Example normalized

In [10]:
landmark = norm_50[20]

xs = [landmark[i][0] for i in range(len(landmark))]
ys = [landmark[i][1] for i in range(len(landmark))]

px.scatter(x=xs, y=ys)

In [20]:
landmark = norm_75[20]

xs = [landmark[i][0] for i in range(len(landmark))]
ys = [landmark[i][1] for i in range(len(landmark))]

px.scatter(x=xs, y=ys)

In [29]:
def get_distances_marks_from_center(landmark: list) -> list:
    distances = list()

    xmean = np.mean(xs)
    ymean = np.mean(ys)

    for mark in landmark:
        x = mark[0]
        y = mark[1]

        distance = pow(pow(x - xmean, 2) + pow(y - ymean, 2), 1/2)

        distances.append(distance)

    return distances

In [31]:
mark_distances_50 = [get_distances_marks_from_center(landmark=landmark) for landmark in norm_50]

In [39]:
mark_distances_50 = [get_distances_marks_from_center(landmark=landmark) for landmark in norm_50]

average_distances = list()
std_distances = list()

for i in range(21):
    zs = [mark[i] for mark in mark_distances_50]
    
    average_distance = np.mean(zs)
    std_distance = np.std(zs)

    average_distances.append(average_distance)
    std_distances.append(std_distance)

In [40]:
mark_distances_75 = [get_distances_marks_from_center(landmark=landmark) for landmark in norm_75]

average_distances = list()
std_distances = list()

for i in range(21):
    zs = [mark[i] for mark in mark_distances_75]
    
    average_distance = np.mean(zs)
    std_distance = np.std(zs)

    average_distances.append(average_distance)
    std_distances.append(std_distance)

In [41]:
mark_distances_100 = [get_distances_marks_from_center(landmark=landmark) for landmark in norm_100]

average_distances = list()
std_distances = list()

for i in range(21):
    zs = [mark[i] for mark in mark_distances_100]
    
    average_distance = np.mean(zs)
    std_distance = np.std(zs)

    average_distances.append(average_distance)
    std_distances.append(std_distance)

In [55]:
px.scatter(x=zs)

In [53]:
zs = [mark[0] for mark in mark_distances_100]
px.scatter(x=zs)
stats.shapiro(zs)

ShapiroResult(statistic=0.8539389371871948, pvalue=6.70783359413285e-11)

In [54]:
for i in range(21):
    zs = [mark[i] for mark in mark_distances_50]
    stat, p = stats.shapiro(zs)

    if p <= 0.05:
        print(f"Mark {i} is not normal")

Mark 0 is not normal
Mark 1 is not normal
Mark 2 is not normal
Mark 3 is not normal
Mark 4 is not normal
Mark 5 is not normal
Mark 6 is not normal
Mark 7 is not normal
Mark 8 is not normal
Mark 9 is not normal
Mark 10 is not normal
Mark 11 is not normal
Mark 12 is not normal
Mark 13 is not normal
Mark 14 is not normal
Mark 15 is not normal
Mark 16 is not normal
Mark 17 is not normal
Mark 18 is not normal
Mark 19 is not normal
Mark 20 is not normal


# Calculating the Variance of each landmark along the x and y axis.

In [11]:
def get_axis_variance_landmark(landmark: list, axis: int) -> float:
    zs = [landmark[i][axis] for i in range(len(landmark))]
    var = np.var(zs)

    return var

In [12]:
def get_landmarks_variance(landmarks: list) -> list:
    xs = list()
    ys = list()

    for landmark in landmarks:
        xs.append(get_axis_variance_landmark(landmark=landmark, axis = 0))
        ys.append(get_axis_variance_landmark(landmark=landmark, axis = 1))

    return [xs, ys]

## Calcutating the mean and stardart deviation for each distance

In [13]:
xs_50, ys_50 = get_landmarks_variance(landmarks=norm_50)

x_mean_50 = np.mean(xs_50)
y_mean_50 = np.mean(ys_50)

x_std_50 = np.std(xs_50)
y_std_50 = np.std(ys_50)

In [14]:
xs_75, ys_75 = get_landmarks_variance(landmarks=norm_75)

x_mean_75 = np.mean(xs_75)
y_mean_75 = np.mean(ys_75)

x_std_75 = np.std(xs_75)
y_std_75 = np.std(ys_75)

In [15]:
xs_100, ys_100 = get_landmarks_variance(landmarks=norm_100)

x_mean_100 = np.mean(xs_100)
y_mean_100 = np.mean(ys_100)

x_std_100 = np.std(xs_100)
y_std_100 = np.std(ys_100)

# Proving that the variances follow a normal distribution

In [16]:
px.scatter(x=[xs_50, xs_75, xs_100])

In [17]:
stat, p = stats.shapiro(xs_50)
print(f"50cm --- stat={stat}   prob={p}")

50cm --- stat=0.9844847917556763   prob=0.09025584906339645


In [18]:
stat, p = stats.shapiro(xs_75)
print(f"75cm --- stat={stat}   prob={p}")

75cm --- stat=0.9050593972206116   prob=2.578703828248763e-08


In [19]:
stat, p = stats.shapiro(xs_100)
print(f"100cm --- stat={stat}   prob={p}")

100cm --- stat=0.9582333564758301   prob=0.00016905792290344834


# Hypothesis definition
- Null hypothesis: the distance affects the gesture
- Research hypothesis: regardless of the distance, the gesture is still statistically the same
- alpha = 95%